In [1]:
import numpy as np
import pandas as pd
from math import sqrt
from operator import add
from functools import reduce
def raw_data(path):
  data = pd.read_csv(path)
  return data.values

In [2]:
import math

def gaussian(u, sigma):
   #   if  np.abs(u) <=  np.abs(1/2):
      return (1/(math.sqrt(2*math.pi) * sigma)) * math.exp(-(u**2)/(2*sigma**2))
   #   else:
      #   return 0

In [3]:
def l2(d1, d2):
    return sqrt(reduce(add, map(lambda y1, y2: (y1 - y2) ** 2, d1, d2)))

In [5]:
from scipy.stats import multivariate_normal

def truePDF(data, mu, cov, per):
    true_pdf = 0
    for i in range(len(mu)):
        true_pdf += (per[i] * multivariate_normal(mu[i], cov[i]).pdf(data))
    return true_pdf

In [13]:
def kernel(dataPoint, x, h, sigma):
    dim = len(dataPoint)
    prod = 1
    for j in range(0, dim):
        prod *= gaussian((x[j]-dataPoint[j])/h, sigma)
    return prod

def kFold(data_length, k):
    folds = []
    fold_size = data_length/k
    for i in range(k):
        folds.append([int(i*fold_size),int(((i+1)*fold_size)) - 1])
    return folds

def generateData(data, size):
    x = np.linspace(np.amin(data[:, 0]), np.amax(
        data[:, 0]), size).reshape(-1, 1)
    y = np.linspace(np.amin(data[:, 1]), np.amax(
        data[:, 1]), size).reshape(-1, 1)
    xx, yy = np.meshgrid(x, y)
    X_2d = np.concatenate(
        [xx.ravel().reshape(-1, 1), yy.ravel().reshape(-1, 1)], axis=1)
    return xx, yy, X_2d

def KDE(data, mu, cov, per, h_min, h_max, h_step, sigma, k, sample_size):
    
    folds = kFold(len(data), k)
    np.random.shuffle(data)
    min_error = 1e9
    best_h = 0

    for h in np.arange(h_min, h_max, h_step):
        print("h: ", h)
        folds_h = []
        MSE = []
        for i, index in enumerate(folds):
            print("fold: ", i)
            xx, yy, X_2d = generateData(data[index[0]:index[1],:], sample_size)
            data_2d = data[index[1]+1:]
            N = np.size(X_2d, 0)
            d = np.size(data_2d, 1)  

            est_density = []  
            for x in X_2d:
                px = 1/N * 1/(h**d) * np.sum([kernel(dataPoint, x, h, sigma) for dataPoint in data_2d])
                est_density.append(px) 
            true_density = truePDF(X_2d,mu, cov, per)
            est_density = np.array(est_density)

            error = l2(true_density, est_density)
            MSE.append(error)
        mse = np.sum(MSE) / k
        print("h =", round(h, 2), ": l2 =", mse)
        if mse < min_error:
            min_error = mse
            best_h = h
    return best_h, min_error

In [10]:
data_2s = raw_data('dataset.csv')
data = data_2s[:,:-1]
print("2D Data: \n", data.shape)


2D Data: 
 (1500, 2)


In [11]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fold = 5
sigmas = [0.6]

mu = np.array([[2, 5],
               [8, 1],
               [5, 3]])
cov = np.array([[[2, 0], [0, 2]],
                  [[3, 1], [1, 3]],
                  [[2, 1], [1, 2]]])
per = [1/3,1/3,1/3]
sample_size = 20

In [15]:

for sigma in sigmas:
    print("sigma: ",sigma)
    best_h, min_error = KDE(data, mu, cov, per, 0.3, 0.9, 0.1, sigma, fold, sample_size)
    print("Best H: ", best_h)
print("End Gaussian KDE")


sigma:  0.6
h:  0.3
fold:  0
fold:  1
fold:  2
fold:  3
fold:  4
h = 0.3 : l2 = 0.22564840173786002
h:  0.4
fold:  0
fold:  1
fold:  2
fold:  3
fold:  4
h = 0.4 : l2 = 0.20523257557791857
h:  0.5
fold:  0
fold:  1
fold:  2
fold:  3
fold:  4
h = 0.5 : l2 = 0.1944451659285239
h:  0.6000000000000001
fold:  0
fold:  1
fold:  2
fold:  3
fold:  4
h = 0.6 : l2 = 0.18768458716042072
h:  0.7000000000000002
fold:  0
fold:  1
fold:  2
fold:  3
fold:  4
h = 0.7 : l2 = 0.18291737187981483
h:  0.8000000000000003
fold:  0
fold:  1
fold:  2
fold:  3
fold:  4
h = 0.8 : l2 = 0.1792491548044564
h:  0.9000000000000001
fold:  0
fold:  1
fold:  2
fold:  3
fold:  4
h = 0.9 : l2 = 0.17625023516897304
Best H:  0.9000000000000001
End Gaussian KDE
